# Enunciado


1. Pretende-se um protótipo protocolo $\,{N\choose{N-1}}\,$-OT, usando a abordagem $\,\mathsf{LPN}\,$ “Learning Parity with Noise” +Capítulo 6d:  Oblivious Linear Evaluation para mensagens de $\,n\,$ bytes (i.e. $\,8\times n\,$bits) que possa ser convertido para mensagens $\,m\in \mathbb{F}_p^n\,$ (vetores de $\,n\,$componentes no corpo finito  $\,\mathbb{F}_p\,$). Para isso
    1. Implemente um protótipo do protocolo $\,\mathsf{LPN}$ $\,{N\choose{N-1}}$-OT  para mensagens de $\,n\,$ bytes (i.e. $\,8\times n\,$bits). Ver +Capítulo 6d:  Oblivious Linear Evaluation .
    2. Codificando os elementos de um corpo primo $\;\mathbb{F}_p\;$ em “arrays” de “bytes” , converta a solução anterior num protocolo $\,{N\choose{N-1}}$-OT em que as mensagens são  vetores $\,\mathbb{F}_p^\ell\,$.

----------

# PARTE A

## Definição de variáveis

In [1]:
p = 2  
F = GF(p)
n = 8
V = VectorSpace(F, n)  # Espaço vetorial F_p^n
lambda_security = 128
V

Vector space of dimension 8 over Finite Field of size 2

In [2]:
m1 = V([1, 3, 5, 6, 2, 0, 4, 1])  # Exemplo de vetor em F_p^n
print(m1)
m2 = V.random_element()  # Gera um vetor aleatório em F_p^n
print(m2)

(1, 1, 1, 0, 0, 0, 0, 1)
(0, 0, 0, 1, 0, 1, 1, 1)


In [3]:
soma = m1 + m2  # Soma vetorial em F_p^n
print(soma)
produto_escalar = m1 * m2 
print(produto_escalar)

(1, 1, 1, 1, 0, 1, 1, 0)
1


In [4]:
import binascii

def bytes_to_fp_vetor(byte_string, p, n):
    """Converte uma string de bytes para um vetor em F_p^n, garantindo variação."""
    F = GF(p)
    V = VectorSpace(F, n)
    byte_array = bytearray(byte_string)
    # Se a mensagem for menor que n, preenche com zeros
    if len(byte_array) < n:
        byte_array += bytes(n - len(byte_array))
    # Pega os primeiros n bytes e aplica mod p
    valores = [F(b % p) for b in byte_array[:n]]
    return V(valores)

# Exemplo: Converter a string "hello!!!" para um vetor em F_2^8
msg_bytes = b"msg001"
msg_fp = bytes_to_fp_vetor(msg_bytes, p, n)
print(msg_fp)

(1, 1, 1, 0, 0, 1, 0, 0)


---------------

## Gerador de Bernoulli

O problema LPN (Learning Parity with Noise) baseia-se na dificuldade computacional de resolver sistemas lineares corrompidos por um erro. Especificamente, em LPN, temos um conjunto de equações da forma:
$$ y=Ax+e $$

onde:

- $A$ é uma matriz binária $m×n$ (geralmente aleatória).

- $x$ é um vetor secreto de $n$ bits.

- $e$ é um vetor de erro, onde cada entrada é $1$ com uma pequena probabilidade $\epsilon$ (geralmente pequena, como 0.10.1 ou 0.20.2).

- $y$ é o vetor de observações.

O vetor de erro ee segue uma distribuição Bernoulli com parâmetro ϵϵ, ou seja:
$$ e_i∼Bernoulli(\epsilon)$$

Isso significa que cada bit $e_i$ tem probabilidade $\epsilon$ de ser $1$ (ruído) e $1-\epsilon$ de ser $0$ (sem ruído).

A forma mais direta de implementar um gerador de Bernoulli  $\,\mathcal{B}(\epsilon)$  com a precisão de $\,n\,$ bits, é o algoritmo.
                            $$\mathcal{B}(\epsilon) \;\equiv\;\vartheta \,w\gets \{0,1\}^n\,\centerdot\,\mathsf{if}\,\;\sum_{i=1}^n\,w_i\,2^{-i}\,\leq\, \varepsilon\;\,\mathsf{then}\,\;1\;\,\mathsf{else}\,\;0$$
Aqui  $\,\hat{w} \equiv \sum_{i=1}^n\,2^{-i}\,w_i\,$ é o designado racional de Lebesgue  determinado pela string de bits $\,w\,$.  Em muitos CPU’s ,  $\,\hat{w}\,$ pode ser calculado em tempo constante ; por isso, este é um processo usual para gerar uniformemente racionais no intervalo $\,[0\,,\,1]$.

In [5]:
import random

def bernoulli(epsilon, n=53):
    """
    Gera uma amostra de Bernoulli B(epsilon) usando a construção de Lebesgue.
    - epsilon: parâmetro da distribuição de Bernoulli (0 < epsilon < 1)
    - n: número de bits para a precisão (default: 53, precisão de um double)
    """
    # Gera a string de bits aleatórios {0,1}^n
    w = [random.randint(0, 1) for _ in range(1, n+1)]
    
    # Calcula o racional de Lebesgue
    w_hat = sum(w[i-1] * 2^(-i) for i in range(1,n+1))
    
    return 1 if w_hat <= epsilon else 0

def bernoulli_lambda(epsilon, n=53, lambda_ = lambda_security):
    return [bernoulli(epsilon,n) for _ in range(lambda_)]
    
# Teste com epsilon = 0.1
epsilon = 0.1
print(bernoulli_lambda(epsilon))

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Learning Party with Noise


O gerador $\,\mathsf{LPN}_{\lambda,\epsilon}(\mathsf{s})\,$, para um segredo $\,\mathsf{s}\gets \mathcal{B}^\lambda\,$,   usa parâmetros $\,\lambda\,$ e $\,\epsilon\,$ definindo-se do seguinte modo


$$\mathsf{LPN}_{\lambda,\epsilon}(\mathsf{s})\;\equiv\quad\vartheta\,{a}\gets\mathcal{B}^\lambda\,\centerdot\,e\gets \mathcal{B}(\varepsilon)\,\centerdot\,\vartheta\,t\gets \mathsf{s}\cdot a + e\,\centerdot\,\langle\,a,t\,\rangle$$ 
        

Aqui  $\,\mathsf{s}\cdot a\,\equiv\,\sum_i\,\mathsf{s}_i\times a_i\;$ denota o produto interno dos dois vetores.

In [6]:
def LPN_generator(lambda_, epsilon,s):
    """Gera um par (a, t) segundo o protocolo LPN."""
    
    # Vetor gerado a partir do gerador de Bernouli
    a = bernoulli_lambda(epsilon)
    
    # Erro e seguindo uma distribuição de Bernoulli B(epsilon)
    e = bernoulli(epsilon)
    t = 0
    # Computa t = s ⋅ a + e onde s.a = sum_i(s_i x a_i)
    for i in range(0,lambda_security):
        t += s[i] * a[i]
    t = t % 2
    return a, t

# Exemplo de uso com lambda = 10 e epsilon = 0.1
epsilon = 0.1
# Segredo s gerado a partir do gerador de Bernouli
secret = bernoulli_lambda(epsilon)
a, t = LPN_generator(lambda_security, epsilon,secret)

print("Vetor a:", a)
print("Valor t:", t)

Vetor a: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
Valor t: 0


## Operação Choose(b)

Definindo primeiramente funções auxiliares:

In [7]:
import hashlib

def xof(seed: bytes, nbytes: int):
    shake = hashlib.shake_128()
    shake.update(seed)
    return shake.digest(nbytes)  # retorna nbytes de saída
    
def xof_bits(seed: bytes, nbits: int):
    nbytes = (nbits + 7) // 8
    output = xof(seed, nbytes)
    bits = []
    for byte in output:
        for i in range(8):
            bits.append((byte >> (7 - i)) & 1)
            if len(bits) == nbits:
                return bits
                
def bytes_to_bits(byte_string):
    bits = []
    for byte in byte_string:
        for i in range(8):
            bits.append((byte >> i) & 1)
    return bits

def bits_to_bytes(bits):
    byte_array = bytearray()
    for i in range(0, len(bits), 8):
        byte = 0
        for j in range(8):
            if i + j < len(bits):
                byte |= bits[i + j] << j
        byte_array.append(byte)
    return bytes(byte_array)

E definindo as classes `Sender` e `Receiver`:

### Classe Sender

In [8]:
class Sender:
    def __init__(self, alpha, l, xof_name="shake128"):
        self.alpha = alpha
        self.l = l
        self.xof_name = xof_name
        self.seed = f"{alpha}:{l}".encode()  # usa α e ℓ como seed
        self.criterion = None
        self.pks = []
        self.N = 100        
        self.mensagens = self.gerar_mensagens()
        self.r = []
        self.criptogramas = []
        self.t = 50 #Número de criptogramas a serem feitos para enviar ao Receiver
    
    def gerar_mensagens(self):
        n_bytes = 6
        return [f"msg{i:03d}".encode().ljust(n_bytes, b'\x00')[:n_bytes] for i in range(self.N)]

    def get_vector_ai_ui(self, i, lambda_):
        """Retorna (a_i, u_i) ∈ F₂^λ × F₂, para índice i"""
        # gera (λ + 1) bits para cada i
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]  # bit extra
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        """Gera todos os pares (a_i, u_i) com base na XOF"""
        self.criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def receive_and_verifypks(self, pks):
        self.pks = pks
        for i in range(self.l):
            soma = sum(pks[i][k] for k in range(self.N)) % 2
            _, u_i = self.criterion[i]  # a_i não é necessário aqui
            if soma != u_i:
                print(f"FALHA na linha i={i}: soma={soma}, u_i={u_i}")
                return False
        print("Verificação OK: todas as somas coincidem com u_i")
        return True

    def gerar_r(self, delta, p=0.1):
        while True:
            r = [bernoulli(p) for _ in range(self.l)]
            if sum(r) <= delta:
                self.r = r
                return

    def calcular_criptogramas(self):
        for _ in range(self.t):
            self.gerar_r(128)
            a = [0] * lambda_security
            for i in range(self.l):
                if self.r[i] == 1:
                    a_i, _ = self.criterion[i]
                    a = [(a[j] + a_i[j]) % 2 for j in range(lambda_security)]
    
            criptogramas_k = []
            for k in range(self.N):
                msg_bits = bytes_to_bits(self.mensagens[k])  # Lista de bits da mensagem
                c_k = []
                for bit in msg_bits:
                    soma = sum(self.r[i] * self.pks[i][k] for i in range(self.l)) % 2
                    c_k.append((bit + soma) % 2)
                criptogramas_k.append(c_k)
            self.criptogramas.append((a, criptogramas_k))
            
    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.xof_name: ",self.xof_name)
        print("self.seed: ",self.seed)
        print("self.criterion: ",self.criterion)

### Classe Receiver

In [9]:
class Receiver:
    def __init__(self, eps, lambda_=128):
        self.alpha = None
        self.l = None
        self.eps = eps
        self.secrets = []
        self.criterion = None
        self.N = 100
        self.t = None
        self.b = 0 # Não quero a primeira mensagem do Sender 

    def receive_alpha_l(self, alpha, l):
        self.alpha = alpha
        self.l = l
        self.seed = f"{alpha}:{l}".encode()
        self.criterion = self.get_criterion_sequence(lambda_security)

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        return [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def generate_N_secrets(self):
        self.secrets = []
        for k in range(self.N):
            if k == self.b:
                self.secrets.append(None)  # s_b = ⊥
            else:
                s_k = bernoulli_lambda(self.eps, lambda_security)
                print(f"s_{k} = {s_k}") 
                self.secrets.append(s_k)

    def generate_pks(self):
        # Inicializar t[i][k]
        self.t = [[0 for _ in range(self.N)] for _ in range(self.l)]  
        for i in range(self.l):
            a_i, u_i = self.criterion[i]
            soma = 0
            for k in range(self.N):
                if k != self.b:
                    s_k = self.secrets[k]  # vetor em F₂^λ
                    dot = sum([a_i[j] * s_k[j] for j in range(lambda_security)]) % 2  # produto escalar mod 2
                    e = bernoulli(self.eps)  # bit de erro
                    t_ik = (dot + e) % 2
                    self.t[i][k] = t_ik
                    soma = (soma + t_ik) % 2
                else:
                    self.t[i][k] = None  # inicializa como None por agora
            
            # Calcula t_{i,b} como complemento para somar a u_i
            self.t[i][self.b] = (u_i - soma) % 2

    def recuperar_mensagens(self, a, c):
        mensagens_recuperadas = []
        for k in range(self.N):
            if k == self.b:
                mensagens_recuperadas.append(None)
            else:
                s_k = self.secrets[k]
                a_dot_s = sum(a[j] * s_k[j] for j in range(lambda_security))
                bits_recuperados = [(c[k][i] + a_dot_s) % 2 for i in range(len(c[k]))]
                msg_bytes = bits_to_bytes(bits_recuperados)
                mensagens_recuperadas.append(msg_bytes)
        return mensagens_recuperadas

    def recuperar_mensagens_maioritarias(self, lista_criptogramas):
        t = len(lista_criptogramas)
        resultados_por_k = [[] for _ in range(self.N)]
        
        for i in range(t):
            a, c = lista_criptogramas[i]
            mks = self.recuperar_mensagens(a, c)
            for k in range(self.N):
                resultados_por_k[k].append(mks[k])
        
        mensagens_finais = []
        for k in range(self.N):
            if k == self.b:
                mensagens_finais.append(None)
            else:
                # Encontra a mensagem mais frequente (votação majoritária)
                contagem = {}
                for msg in resultados_por_k[k]:
                    contagem[msg] = contagem.get(msg, 0) + 1
                mensagem_mais_votada = max(contagem.items(), key=lambda x: x[1])[0]
                mensagens_finais.append(mensagem_mais_votada)
        return mensagens_finais

    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.lambda: ",lambda_security)
        print("self.eps: ",self.eps)
        print("self.secrets: ",self.secrets)
        print("self.criterion: ",self.criterion)
        
    def print_secrets(self):
        print("[")
        for k in range(N):
            if k == self.b:
                print("None")
            else:
                print(self.secrets[k])
        print("]")

**Nota**: As classes já possuem todas as operações que são precisas neste ponto mas será explicado devidamente o processo a partir deste ponto.

---------

Por passos iremos definir o Choose(b):

"O sender  escolhe o par $\,(\alpha,\ell)\,$ e a função XOF e envia essa informação para o sender;  esta informação determina completamente   a sequência $\;\{\langle\,a_i,u_i\,\rangle\}_{i=1}^\ell\,$  que passa a  formar o “oblivious criterion” ; ambos os agentes podem construir estes elementos."

In [10]:
# Setup
sender = Sender(alpha="alpha123", l=10)
sender.get_criterion_sequence(lambda_security)
#sender.print_info()
receiver = Receiver(eps=0.1)
receiver.receive_alpha_l("alpha123", 10)
#receiver.print_info()

# Verificação: os critérios devem bater
if sender.criterion == receiver.criterion:
    print("Critérios coincidem")
else:
    print("Critérios não coincidem")

Critérios coincidem


---

o receiver gera $\,N\,$ segredos $\,\mathsf{s}_k\gets \mathcal{B}^\lambda\,$,  se $k\neq b\,$, e $\,\mathsf{s}_b \gets \bot\,$

In [11]:
receiver.generate_N_secrets()
#print("Segredos do receiver:")
#receiver.print_secrets()

s_1 = [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
s_2 = [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s_3 = [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

---------

Para todo $k \in [N]$ e todo $\,i\in [\ell]\,$ , calcula $\,t_{k,i}$ da seguinte forma

$$t_{i,k}\;\gets\;\left\{\begin{array}{lcl}\vartheta\,e_{i,k}\gets \mathcal{B(\epsilon)}\,\centerdot\, a_i\cdot \mathsf{s}_k + e_{i,k} & \text{se} & k\neq b \\ u_i - \sum_{j\neq b}\,t_{i,j} &\text{se}& k=b\end{array}\right.$$

Regista esta informação na sua memória.  
Construímos, para cada $\,i\in[\ell]\,$ , um vetor em $\,\mathcal{B}^N$

$$\mathsf{t}_i\,\equiv\,\{t_{i,k} \;|\;k\in [N]\}\,$$

e envia-os para o sender  como chaves públicas.

In [12]:
receiver.generate_pks()

--------

c .  o sender   recolhe todas os vetores de chaves públicas $\,\mathsf{t_i}\,$ e verifca as igualdades 
                                                         $$\sum_{k\in [N]}\,\mathsf{t}_{i,k}\;=\; u_i$$

Se, para algum $\,i\in[\ell]\,$  a igualdade não se verifica então termina em falha.
Se se verificar a igualdade então  regista todos os $\,\mathsf{t}_i\,$ na sua memória para transferência futura.

In [13]:
sender.receive_and_verifypks(receiver.t)

Verificação OK: todas as somas coincidem com u_i


True

## Operação $\mathsf{Transfer}(m_0,m_1,\cdots,m_{N-1})$

Por passos:

1. O **sender** conhece as mensagens $m_k \in \mathbb{F}_2$, $k \in [N]$ e, para cada $i \in [\ell]$ as chaves públicas $t_i$
Para as cifrar gera aleatoriamente uma sequências de bits $\{r_i \leftarrow \mathcal{B} \}_{i=0}^{\ell}$ com um peso de Hamming (número de bits 1) limitado a um parâmetro $\delta$, e calcula

$$a \leftarrow \sum_i r_i . e_{i,k} \ \ \ \ \ , \ \ \ \ \ c_k \leftarrow m_k + \sum_i r_i . t_{i,k}$$

para todos os $ k \in [N]$
O criptograma é o tuplo $\langle a,c_o,...,c_{N-1} \rangle$ que é enviado para o **receiver**.

In [14]:
#Gerar aletoriamenete a sequência de bits ri
sender.calcular_criptogramas()
print(len(sender.criptogramas))

50


2. O receiver  conhece os segredos $\,\mathsf{s}_k\,$  para todo $\,k\in[N]$ . Sabe que $\,\mathsf{s}_b = \bot\,$e que para todo $\,k\neq b\,$  pode calcular $\,a\cdot \mathsf{s}_k\,$. Sabe também que se verifica, para todo $\,k\neq b\,$, a relação
                            $$m_k \;=\; c_k + (a\cdot\mathsf{s}_k) + \mathsf{error}_k$$
sendo $\,\mathsf{error}_k\,=\,\sum_{i}\,r_i\cdot e_{i,k}$  um valor desconhecido (porque o receiver não conhece os $r_i$ )  mas com elevada probabilidade de ser nulo. 
        
Procedendo como no protocolo ${2\choose{}1}$-OT , pode-se reforçar esta probabilidade, iterando ambas as operações $\,t\,$ vezes; para isso cifra-se usando  vetores $\,r_i\in \mathcal{B}^t\,$. As iterações são independentes e podem ser executadas em paralelo. A sender produz $\,t\,$ criptogramas distintos, um por iteraçao. 
O receiver toma este conjunto de criptogramas e calcula, para cada um, um resultado
$$m_k \gets c_k + (a\cdot s_k)\quad$$ para todo $\,k\neq b$
Toma-se como resultado final de $\,m_\kappa\,$ , para cada $k\neq b\,$, o valor em maioria nas diferentes iterações; assim  obtém-se, com elevada probabilidade,  o valor da mensagem inicial.

Finalmente para mensagens $\,\{m_k\}_{k\in[N]}\,$ de comprimento arbitrário, tal como no caso, $\,{2\choose{}1}$OT, usa-se o protocolo de mensagens binárias para cada posição nas mensagens.

In [15]:
mensagens_recuperadas = receiver.recuperar_mensagens_maioritarias(sender.criptogramas)

Verificação se conseguiu decifrar corretamente as mensagens assim como não cifrar corretamente a mensagem de índice b

In [16]:
for i in range(0,len(sender.mensagens)):
    if receiver.b != i:
        if sender.mensagens[i] == mensagens_recuperadas[i]:
            print(f"Mensagem de índice {i} bem decifrada")
        else:
            print(f"Mensagem de índice {i} mal decifrada")
    else:
        if sender.mensagens[i] != mensagens_recuperadas[i]:
            print(f"A mensagem de índice {i} que não era suposto decifrar foi corretamente mal decifrada")

A mensagem de índice 0 que não era suposto decifrar foi corretamente mal decifrada
Mensagem de índice 1 bem decifrada
Mensagem de índice 2 bem decifrada
Mensagem de índice 3 bem decifrada
Mensagem de índice 4 bem decifrada
Mensagem de índice 5 bem decifrada
Mensagem de índice 6 bem decifrada
Mensagem de índice 7 bem decifrada
Mensagem de índice 8 bem decifrada
Mensagem de índice 9 bem decifrada
Mensagem de índice 10 bem decifrada
Mensagem de índice 11 bem decifrada
Mensagem de índice 12 bem decifrada
Mensagem de índice 13 bem decifrada
Mensagem de índice 14 bem decifrada
Mensagem de índice 15 bem decifrada
Mensagem de índice 16 bem decifrada
Mensagem de índice 17 bem decifrada
Mensagem de índice 18 bem decifrada
Mensagem de índice 19 bem decifrada
Mensagem de índice 20 bem decifrada
Mensagem de índice 21 bem decifrada
Mensagem de índice 22 bem decifrada
Mensagem de índice 23 bem decifrada
Mensagem de índice 24 bem decifrada
Mensagem de índice 25 bem decifrada
Mensagem de índice 26 bem 

# Parte B

b. Codificando os elementos de um corpo primo $\;\mathbb{F}_p\;$ em “arrays” de “bytes” , converta a solução anterior num protocolo $\,{N\choose{N-1}}$-OT em que as mensagens são  vetores $\,\mathbb{F}_p^\ell\,$.

In [17]:
p = 257  # Exemplo: primo de 8 bits
l = 10   # Tamanho do vetor F_p^ℓ
lambda_security = 128
F = GF(p)

Definimos as funções auxiliares:

In [18]:
def vector_to_bytes(vector, p):
    """Converte um vetor F_p^ℓ em bytes."""
    byte_array = bytearray()
    for elemento in vector:
        # Assume que elemento está em {0, ..., p-1}
        byte_array.append(elemento % 256)  # 1 byte por elemento (se p ≤ 256)
    return bytes(byte_array)

def bytes_to_vector(byte_string, p, l):
    """Converte bytes de volta para F_p^ℓ."""
    vector = []
    for byte in byte_string[:l]:
        vector.append(byte % p)
    return vector

E as novas classes que irão suportar arrays de bytes:

In [19]:
class Sender:
    def __init__(self, alpha, l, xof_name="shake128"):
        self.alpha = alpha
        self.l = l
        self.xof_name = xof_name
        self.seed = f"{alpha}:{l}".encode()  # usa α e ℓ como seed
        self.criterion = None
        self.pks = []
        self.N = 100        
        self.mensagens = self.gerar_mensagens()
        self.r = []
        self.criptogramas = []
        self.p = 257
        self.t = 50 #Número de criptogramas a serem feitos para enviar ao Receiver
    
    def gerar_mensagens(self):
        return [b''.join(((i + j) % p).to_bytes(2, byteorder='little') for j in range(l))for i in range(self.N)]

    def get_vector_ai_ui(self, i, lambda_):
        """Retorna (a_i, u_i) ∈ F₂^λ × F₂, para índice i"""
        # gera (λ + 1) bits para cada i
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]  # bit extra
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        """Gera todos os pares (a_i, u_i) com base na XOF"""
        self.criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def receive_and_verifypks(self, pks):
        self.pks = pks
        for i in range(self.l):
            soma = sum(pks[i][k] for k in range(self.N)) % 2
            _, u_i = self.criterion[i]  # a_i não é necessário aqui
            if soma != u_i:
                print(f"FALHA na linha i={i}: soma={soma}, u_i={u_i}")
                return False
        print("Verificação OK: todas as somas coincidem com u_i")
        return True

    def gerar_r(self, delta, p=0.1):
        while True:
            r = [bernoulli(p) for _ in range(self.l)]
            if sum(r) <= delta:
                self.r = r
                return

    def calcular_criptogramas(self):
        for _ in range(self.t):
            self.gerar_r(128)
            a = [0] * lambda_security  # Agora em F_p
            for i in range(self.l):
                if self.r[i] == 1:
                    a_i, _ = self.criterion[i]
                    a = [(a[j] + a_i[j]) % self.p for j in range(lambda_security)]
    
            criptogramas_k = []
            for k in range(self.N):
                msg_vector = self.mensagens[k]  # msg_vector é um vetor em F_p^ℓ
                c_k = []
                for elemento in msg_vector:
                    soma = sum(self.r[i] * self.pks[i][k] for i in range(self.l)) % self.p
                    c_k.append((elemento + soma) % self.p)
                criptogramas_k.append(c_k)
            self.criptogramas.append((a, criptogramas_k))
            
    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.xof_name: ",self.xof_name)
        print("self.seed: ",self.seed)
        print("self.criterion: ",self.criterion)

In [20]:
class Receiver:
    def __init__(self, eps, lambda_=128):
        self.alpha = None
        self.l = None
        self.eps = eps
        self.secrets = []
        self.criterion = None
        self.N = 100
        self.t = None
        self.b = 9 # Não quero a décima mensagem do Sender 
        self.p = 257

    def receive_alpha_l(self, alpha, l):
        self.alpha = alpha
        self.l = l
        self.seed = f"{alpha}:{l}".encode()
        self.criterion = self.get_criterion_sequence(lambda_security)

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        return [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def generate_N_secrets(self):
        self.secrets = []
        for k in range(self.N):
            if k == self.b:
                self.secrets.append(None)  # s_b = ⊥
            else:
                s_k = bernoulli_lambda(self.eps, lambda_security)
                print(f"s_{k} = {s_k}") 
                self.secrets.append(s_k)

    def generate_pks(self):
        # Inicializar t[i][k]
        self.t = [[0 for _ in range(self.N)] for _ in range(self.l)]  
        for i in range(self.l):
            a_i, u_i = self.criterion[i]
            soma = 0
            for k in range(self.N):
                if k != self.b:
                    s_k = self.secrets[k]  # vetor em F₂^λ
                    dot = sum([a_i[j] * s_k[j] for j in range(lambda_security)]) % 2  # produto escalar mod 2
                    e = bernoulli(self.eps)  # bit de erro
                    t_ik = (dot + e) % 2
                    self.t[i][k] = t_ik
                    soma = (soma + t_ik) % 2
                else:
                    self.t[i][k] = None  # inicializa como None por agora
            
            # Calcula t_{i,b} como complemento para somar a u_i
            self.t[i][self.b] = (u_i - soma) % 2

    def recuperar_mensagens(self, a, c):
        mensagens_recuperadas = []
        for k in range(self.N):
            if k == self.b:
                mensagens_recuperadas.append(None)
            else:
                s_k = self.secrets[k]  # s_k é um vetor em F_p^λ
                a_dot_s = sum(a[j] * s_k[j] for j in range(lambda_security)) % self.p
                msg_recuperada = [(c[k][i] + a_dot_s) % self.p for i in range(len(c[k]))]
                mensagens_recuperadas.append(msg_recuperada)
        return mensagens_recuperadas

    def recuperar_mensagens_maioritarias(self, lista_criptogramas):
        t = len(lista_criptogramas)
        resultados_por_k = [[] for _ in range(self.N)]
        
        for i in range(t):
            a, c = lista_criptogramas[i]
            mks = self.recuperar_mensagens(a, c)
            for k in range(self.N):
                resultados_por_k[k].append(tuple(mks[k]) if mks[k] is not None else None)  # Convertemos para tupla
        
        mensagens_finais = []
        for k in range(self.N):
            if k == self.b:
                mensagens_finais.append(None)
            else:
                contagem = {}
                for msg in resultados_por_k[k]:
                    if msg is not None:
                        # Usamos tupla como chave (hashable)
                        msg_key = msg
                        contagem[msg_key] = contagem.get(msg_key, 0) + 1
                # Obtém a mensagem mais frequente
                mensagem_mais_votada = max(contagem.items(), key=lambda x: x[1])[0] if contagem else None
                mensagens_finais.append(bytes(mensagem_mais_votada) if mensagem_mais_votada else None)  # Converte de volta para bytes
        
        return mensagens_finais

    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.lambda: ",lambda_security)
        print("self.eps: ",self.eps)
        print("self.secrets: ",self.secrets)
        print("self.criterion: ",self.criterion)
        
    def print_secrets(self):
        print("[")
        for k in range(N):
            if k == self.b:
                print("None")
            else:
                print(self.secrets[k])
        print("]")

In [21]:
sender = Sender(alpha="alpha123", l=10)
sender.get_criterion_sequence(lambda_security)
receiver = Receiver(eps=0.1)
receiver.receive_alpha_l("alpha123", 10)

In [22]:
receiver.generate_N_secrets()
receiver.generate_pks()
sender.receive_and_verifypks(receiver.t)
sender.calcular_criptogramas()
mensagens_decodificadas = receiver.recuperar_mensagens_maioritarias(sender.criptogramas)

s_0 = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
s_1 = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
s_2 = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0

In [23]:
for i in range(0,len(sender.mensagens)):
    print(f"{sender.mensagens[i]} // {mensagens_recuperadas[i]}")

b'\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00' // None
b'\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00' // b'msg001'
b'\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00' // b'msg002'
b'\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00' // b'msg003'
b'\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00' // b'msg004'
b'\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00' // b'msg005'
b'\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00' // b'msg006'
b'\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00' // b'msg007'
b'\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00' // b'msg008'
b'\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00' // b'msg009'
b'\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00' // b'

In [24]:
for i in range(0,len(sender.mensagens)):
    if receiver.b != i:
        if sender.mensagens[i] == mensagens_decodificadas[i]:
            print(f"Mensagem de índice {i} bem decifrada")
        else:
            print(f"Mensagem de índice {i} mal decifrada")
    else:
        if sender.mensagens[i] != mensagens_decodificadas[i]:
            print(f"A mensagem de índice {i} que não era suposto decifrar foi corretamente mal decifrada")

Mensagem de índice 0 bem decifrada
Mensagem de índice 1 bem decifrada
Mensagem de índice 2 bem decifrada
Mensagem de índice 3 bem decifrada
Mensagem de índice 4 bem decifrada
Mensagem de índice 5 bem decifrada
Mensagem de índice 6 bem decifrada
Mensagem de índice 7 bem decifrada
Mensagem de índice 8 bem decifrada
A mensagem de índice 9 que não era suposto decifrar foi corretamente mal decifrada
Mensagem de índice 10 bem decifrada
Mensagem de índice 11 bem decifrada
Mensagem de índice 12 bem decifrada
Mensagem de índice 13 bem decifrada
Mensagem de índice 14 bem decifrada
Mensagem de índice 15 bem decifrada
Mensagem de índice 16 bem decifrada
Mensagem de índice 17 bem decifrada
Mensagem de índice 18 bem decifrada
Mensagem de índice 19 bem decifrada
Mensagem de índice 20 bem decifrada
Mensagem de índice 21 bem decifrada
Mensagem de índice 22 bem decifrada
Mensagem de índice 23 bem decifrada
Mensagem de índice 24 bem decifrada
Mensagem de índice 25 bem decifrada
Mensagem de índice 26 bem 